### pre-process the JMA hindcasts, step 1, read grib, change variable and dimension names and saves to netcdf 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from datetime import datetime, timedelta

In [3]:
import pygrib
import numpy as np

In [4]:
import xarray as xr; print(xr.__version__)

0.14.0


In [5]:
import dask 

In [6]:
import os

In [7]:
import pathlib

In [8]:
var = 'PRECIP'

In [9]:
model = 'JMA'

In [10]:
dpath = pathlib.Path(f'/home/nicolasf/drives/auck_projects/END19101/Working/data/hindcasts/{model}/{var}/')

In [11]:
lfiles = list(dpath.glob("*.grb2"))

In [12]:
len(lfiles)

864

### loops over the files and process 

In [13]:
for fname in lfiles: 

    print(f"processing {fname}")

    out_fname = fname.name.replace('.grb2','.nc')
    out_fname = fname.parent / out_fname
    init_date = datetime.strptime(fname.name.split('.')[-2], "%Y%m%d")
    
    grbs = pygrib.open(str(fname))

    analDates = []
    validDates = []
    validityDates = []
    data = []

    for g in grbs: 
        data.append(g.values)
        analDates.append(g.analDate)
        validDates.append(g.validDate)
        validityDates.append(datetime.strptime(str(g.validityDate), "%Y%m%d"))   

    data = np.array(data)

    data_reshaped = np.reshape(data, (len(np.unique(validDates)), data.shape[0] // len(np.unique(validDates)), data.shape[-2], data.shape[-1]))

    data_reshaped = data_reshaped[np.newaxis, ...]

    lats,lons = g.latlons()

    lats = lats[:,0]
    lons = lons[0,:]

    nmembers =  g.numberOfForecastsInEnsemble

    if nmembers != data_reshaped.shape[2]:
        print('oups, data shape doesnt match the number of ensemble members')
    if len(np.unique(validDates)) != 7: 
        print('oups, issue with the number of steps')

    d = {}

    d['time'] = (('time'), np.array(init_date).reshape(1,))
    d['step'] = (('step'), [1,2,3,4,5,6,7])
    d['latitude'] = (('latitude'), lats)
    d['longitude'] = (('longitude'), lons)
    d['number'] = (('number'), list(range(0, nmembers)))
    d[g.shortName.lower()] = (('time', 'step', 'number', 'latitude','longitude'), data_reshaped)
    

    dset = xr.Dataset(d)

    ### mask the land values (np.nan)
    
    dset[g.shortName.lower()] = dset[g.shortName.lower()].where(dset[g.shortName.lower()] < 9999, np.nan)

    dset = dset.rename({g.shortName.lower():'precip'})
    
    dset.to_netcdf(out_fname, unlimited_dims='time')
#     dset.to_netcdf(out_fname, encoding={g.shortName.lower():{'dtype':'float64'}})

    dset.close()


processing /home/nicolasf/drives/auck_projects/END19101/Working/data/hindcasts/JMA/PRECIP/surf_Prr_mon.19790116.grb2
processing /home/nicolasf/drives/auck_projects/END19101/Working/data/hindcasts/JMA/PRECIP/surf_Prr_mon.19790131.grb2
processing /home/nicolasf/drives/auck_projects/END19101/Working/data/hindcasts/JMA/PRECIP/surf_Prr_mon.19790210.grb2
processing /home/nicolasf/drives/auck_projects/END19101/Working/data/hindcasts/JMA/PRECIP/surf_Prr_mon.19790225.grb2
processing /home/nicolasf/drives/auck_projects/END19101/Working/data/hindcasts/JMA/PRECIP/surf_Prr_mon.19790312.grb2
processing /home/nicolasf/drives/auck_projects/END19101/Working/data/hindcasts/JMA/PRECIP/surf_Prr_mon.19790327.grb2
processing /home/nicolasf/drives/auck_projects/END19101/Working/data/hindcasts/JMA/PRECIP/surf_Prr_mon.19790411.grb2
processing /home/nicolasf/drives/auck_projects/END19101/Working/data/hindcasts/JMA/PRECIP/surf_Prr_mon.19790426.grb2
processing /home/nicolasf/drives/auck_projects/END19101/Working/